# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install -U openai

In [ ]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

# Function Calling の実行

In [ ]:
import requests

# 外部関数の定義
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [ ]:
# ツールリストの準備
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "指定された座標の現在の温度を摂氏で取得します。",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

In [ ]:
# メッセージリストの準備
messages = [
    {"role": "user", "content": "東京の気温は？"}
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages,
    tools=tools  # ツール
)
print(response.output_text)

In [ ]:
tool_call = None

# 出力の確認
for item in response.output:
    # function_call
    if item.type == "function_call":
        print("==function_call==")
        print(item)
        tool_call = item

In [ ]:
import json

# 外部関数の実行
args = json.loads(tool_call.arguments)
result = get_weather(args["latitude"], args["longitude"])
print(result)

In [ ]:
# メッセージリストの準備 (外部関数呼び出し結果)
messages2 = [
    {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
]

# 推論の実行
response2 = client.responses.create(
    model="gpt-5",
    input=messages2,
    tools=tools,
    previous_response_id=response.id, # 会話状態管理
)
print(response2.output_text)